In [ ]:
from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
import cv2

from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import imutils

In [ ]:
##################################################### Data cleanup and loading #####################################################

In [ ]:
store_image=[]
train_path='E://handy//ml1//train'
fps=5
train_videos=os.listdir('E://handy//ml1//train//training_videos')
train_images_path=train_path+'/frames'
try:
    os.makedirs(train_images_path)
except FileExistsError:
    pass
    
def store_inarray(image_path):
    image=load_img(image_path)
    image=img_to_array(image)
    image=cv2.resize(image, (227,227), interpolation = cv2.INTER_AREA)
    gray=0.2989*image[:,:,0]+0.5870*image[:,:,1]+0.1140*image[:,:,2]
    store_image.append(gray)


In [ ]:
for video in train_videos:
    os.system( 'ffmpeg -i E://handy//ml1//train//training_videos//{} -r 1/{}  E://handy//ml1//train//frames/%03d.jpg'.format(video,fps))
    images=os.listdir(train_images_path)
    for image in images:
        image_path=train_images_path + '/' + image
        store_inarray(image_path)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def rotate(image, angle=90, scale=1.0):
        w = image.shape[1]
        h = image.shape[0]
        M = cv2.getRotationMatrix2D((w/2,h/2), angle, scale)
        image = cv2.warpAffine(image,M,(w,h))
        
        kernel = np.ones((5, 5), np.uint8)
        image = cv2.dilate(image, kernel, iterations = 1)
        image = cv2.erode(image, kernel, iterations = 1)
        return image
def flip(image, vflip=False, hflip=False):
        if hflip or vflip:
            if hflip and vflip:
                c = -1
            else:
                c = 0 if vflip else 1
            image = cv2.flip(image, flipCode=c)
        return image 
    
def image_agumentation:  
    images=os.listdir(train_images_path)
    for image in images:
            image_path=train_images_path + '/' + image
            img = mpimg.imread(image_path)
            plt.imshow(img)
            plt.show()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            plt.imshow(gray, cmap="gray", vmin=0, vmax=255)
            plt.show()
            img_flip = flip(gray, vflip=True, hflip=False)
            img_rot = rotate(img_flip)
            imgplot = plt.imshow(img_rot, cmap="gray", vmin=0, vmax=255)
            

In [ ]:
##################################################### model training #####################################################

In [ ]:
store_image=np.array(store_image)
a,b,c=store_image.shape

store_image.resize(b,c,a)
store_image=(store_image-store_image.mean())/(store_image.std())
store_image=np.clip(store_image,0,1)
np.save('training.npy',store_image)

In [ ]:
stae_model=Sequential()

stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [ ]:
training_data=np.load('training1.npy')
frames=training_data.shape[2]
frames=frames-frames%10

training_data=training_data[:,:,:frames]
training_data=training_data.reshape(-1,227,227,10)
training_data=np.expand_dims(training_data,axis=4)
target_data=training_data.copy()

epochs=10
batch_size=1

callback_save = ModelCheckpoint("Xmodel.h5", monitor="mean_squared_error", save_best_only=True)

callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

stae_model.fit(training_data,target_data, batch_size=batch_size, epochs=epochs, callbacks = [callback_save,callback_early_stopping])
stae_model.save("Xmodel.h5")

In [ ]:
model = load_model("Xmodel.h5")
model.summary()

In [ ]:
from keras.models import load_model
from firebase import firebase
import cv2
import numpy as np 
import argparse
from PIL import Image
import imutils

sample_anamoly_video1 = "C://Users//Utkarsh//Desktop//1_Trim.mp4"
sample_anamoly_video2 = "C://Users//Utkarsh//Desktop//2.mp4"
sample_normal_video = "C://Users//Utkarsh//Desktop//01.avi"



def mean_squared_loss(x1,x2):
    difference = x1-x2
    a,b,c,d,e = difference.shape
    n_samples = a*b*c*d*e
    sq_difference = difference**2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance/n_samples

    return mean_distance

def smart_cam(sample_video):
    threshold = 0.00057
    model = load_model("Xmodel.h5")

    cap = cv2.VideoCapture(sample_video)
    print("SMART SURVAILLANCE: {}".format(cap.isOpened()))

    while cap.isOpened():
        imagedump = []
        _, frame = cap.read()

        if frame.any()== None:
            break

        for i in range(10):
            _, frame = cap.read()
            image = imutils.resize(frame,width=700,height=600)

            frame = cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
            gray = 0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
            gray = (gray-gray.mean())/gray.std()
            gray = np.clip(gray,0,1)
            imagedump.append(gray)

        imagedump = np.array(imagedump)

        imagedump.resize(227,227,10)
        imagedump = np.expand_dims(imagedump,axis=0)
        imagedump = np.expand_dims(imagedump,axis=4)

        output = model.predict(imagedump)

        loss = mean_squared_loss(imagedump,output)
#         print(loss)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
        print(loss)
        if loss > threshold:
            cv2.putText(image,"Abnormal Event",(100,80),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
            if loss < 0.00058 and loss > 0.00056: 
                cv2.putText(image,"Severity: Moderate",(100,120),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            elif loss > 0.00062:
                cv2.putText(image,"Severity: High",(100,120),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)

        cv2.imshow("video",image)

    cap.release()
    cv2.destroyAllWindows()
    
smart_cam(sample_normal_video)
smart_cam(sample_anamoly_video1)
smart_cam(sample_anamoly_video2)
